# Create and monitor copy collection jobs in Azure Cosmos DB SQL API account

## Login to the Subscription.

In [ ]:
az login

## Set a subscription to be the current active subscription

(needed only if you multiple subscriptions associated with your account)

In [ ]:
az account set --subscription ""

## Specify parameters necessary for the copy  
**Note:** <span style="font-size: 14px;">&nbsp;armlocation is required for private preview az cli extension,&nbsp;</span> _please don't modify it_

In [ ]:
$cosmosdbaccountname = "cosmosDBAccountName"
$rg = "resourceGroupName"
$copyJobName = ""
$sourceContainer = ""
$sourceDb = ""
$destinationContainer = ""
$destinationDb = ""
$location = "centralus"
$armlocation = "centraluseuap"

### Load Data Transfer Private AZ CLI Package

Please ignore below warning and error in the output(this is common if you re-execute the script):

- WARNING: Command group 'config' is experimental and under development
- ERROR: The extension cosmosdb-preview already exists

In [ ]:
az cloud update --endpoint-resource-manager "https://$armlocation.management.azure.com"
az config set defaults.location=$location --local
az extension add --source https://dtsclidist.blob.core.windows.net/cli-private-preview/V2/cosmosdb_preview-0.10.0-py2.py3-none-any.whl -y

## Provision the server-side instances for the copy jobs

In [ ]:
az cosmosdb service create --account-name $cosmosdbaccountname -g $rg --service-name DataTransferService --kind "DataTransferService" --count 1 --size "Cosmos.D4s"

## Create container copy job

In [ ]:
az cosmosdb dts copy -g $rg --job-name $jobName --account-name $cosmosdbaccountname \
    --source-sql-container database=$sourceContainer container=$sourceDb \
    --dest-sql-container database=$destinationContainer container=$destinationDb

## Monitor the progress of the container copy job

In [ ]:
az cosmosdb dts show --account-name  $cosmosdbaccountname -g $rg --job-name $copyJobname

## Delete the server-side instances created for the copy

In [ ]:
az cosmosdb service delete --account-name $cosmosdbaccountname -g $rg --service-name "DataTransferService" --yes